#Second Practical work

This is second practical work skeleton.

It is divided in 3 parts_

1.   Python module load section: load all the needed modules
2.   Google Drive access and mounting
3.   Host load data
4.   GPU load





##Module load section
We will install the pycuda module in the colaboratory environment, to combine python and CUDA in our code. Because pycuda is not available by default, this task can take a few minutes.

As well we load numpy and matplotlib modules.

In [ ]:
!pip install pycuda

We import the Python modules which we will use in the notebook (numpy, pycuda and time module) 

In [ ]:
import  numpy  as  np
import  pycuda.autoinit
from    pycuda.compiler import SourceModule
import  pycuda.driver as  drv
import  pycuda.gpuarray as  gpuarray
from sys import getsizeof
import time

In [ ]:
%pylab inline
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cProfile

##Mount Google Drive and access grant
We use Google API to allow access to our Google Drive.

For security and privacy reasons, this permission is temporally, and will be requested every time you restart the notebook. 

Your Google Drive will be mounted in the "/content/drive" folder. The first Folder will be "My Drive".

You have to copy your images to a folder in the "My Drive" folder in your Google Drive.

To check your available files, use the cell indicated as "**List Files in Folder**"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from os.path import join
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
MYDRIVE="/content/drive/My Drive/"

**List Files in Folders**

In [ ]:
!ls /content/drive/My\ Drive/images

We define a variable IMAGE which is constructed joining the "MYDRIVE" varible plus the final path to your image.

In [ ]:
IMAGE=MYDRIVE + 'images/digits.jpg'

In [ ]:
print("Image to load: {0}".format(IMAGE))

Loads an image in a numpy variable, and show.

In [ ]:
image=np.array(Image.open(IMAGE))
plt.figure()
plt.imshow(image)

#FILTER KERNEL

Here you have to define your image_filter GPU kernel.

Recover the image shape, and select the two first items (rows and columns).

In [ ]:
image_shape=image.shape[0:2]

Set block size and grid size.

In [ ]:
block_s=16
block_size=(block_s,block_s,1)
grid_size=(int(np.ceil(image_shape[1]/block_s)),int(np.ceil(image_shape[0]/block_s)))

Allocates the target image memory.

In [ ]:
result_image=np.zeros(image.shape,dtype=np.uint8)

In [ ]:
depth = image.shape[2]


In [ ]:
print(image.shape)

Here we defines our sample filter masks.

In [ ]:
filter1=np.array([
    [0,0,0,0,0],
    [0,0,0,0,0],
    [0,0,1,0,0],
    [0,0,0,0,0],
    [0,0,0,0,0]
])
filter2=np.array([[0.5, 0 , -0.5]])
filter3=np.array([[0.5],[0],[-0.5]])

filter4=np.array([
    [1,0,-1],
    [2,0,-2],
    [1,0,-1]
])
filter5=np.array([
    [0.00078633,0.00655965,0.01330373,0.00655965,0.00078633],
    [0.00655965,0.05472157,0.11098164,0.05472157,0.00655965],
    [0.01330373,0.11098164,0.22508352,0.11098164,0.01330373],
    [0.00655965,0.05472157,0.11098164,0.05472157,0.00655965],
    [0.00078633,0.00655965,0.01330373,0.00655965,0.00078633]
])

We select the filter mask, and upload to the GPU device.

In [ ]:
filter_mask=filter5.astype(np.float32)
filter_mask_gpu=gpuarray.to_gpu(filter_mask)

We write our kernel function. NOTE: Moved to lower in the notebook so that we could have block_size, NumRowsFilter, and NumColsFilter be constants. This is required by the compiler for allocating the shared memory.

In [ ]:
image_filter_tmp  =  """

__global__ void image_filter( float * image,          //Source GPU array floating point 32 bits,
		                          float * filter_mask,    //Filter Mask GPU array 2D floating point 32 bits
                              float * filtered_image, //Target GPU array 2D floating point 32 bits,
                              int NumRowsImg,         //Image Numrows,
                              int NumColsImg,         //Int32 Image Numcolumns,
                              int NumRowsFilter,       //Int32 Image NumRows filter mask,
                              int NumColsFilter       //Int32 Image NumCols filter mask
                              ) 
{ 
    // Shared memory for image tile
    __shared__ float imageTile[%(BLOCK_SIZE)s+%(FILTER_ROWS)s-1][%(BLOCK_SIZE)s+%(FILTER_COLS)s-1];
    __shared__ float filterShared[%(FILTER_ROWS)s][%(FILTER_COLS)s];

    // Fill tiled memory for image
    unsigned int offsetRows = NumRowsFilter / 2;
    unsigned int offsetCols = NumColsFilter / 2;

    // idxY is the index of the row in our target image we are computing with this thread
    // idxX is the index of the col in our target image we are computing with this thread
    unsigned int idxY = blockIdx.y*blockDim.y+threadIdx.y; //With this we calculate the row address in target matrix
    unsigned int idxX = blockIdx.x*blockDim.x+threadIdx.x;
    
    // ---FILL TILE INNER---
    // ---FILL TILE INNER---
    // ---FILL TILE INNER---
    // ---FILL TILE INNER---
    // ---FILL TILE INNER---
    
    // Fills the inner part of the tile (direct copy over from original image taking into account blocks that might exceed image matrix borders)
    if ((idxY<NumRowsImg) && (idxX<NumColsImg)){
      unsigned int vIdx=idxY*NumColsImg+idxX;
      imageTile[offsetRows+threadIdx.y][offsetCols+threadIdx.x] = image[vIdx];
    } else {
      unsigned int r = idxY;
      unsigned int c = idxX;
      if (r >= NumRowsImg) {
        r = NumRowsImg - 1;
      }
      if (c >= NumColsImg) {
        c = NumColsImg - 1;
      }
      //calculate the index in the original matrix
      unsigned int vIdxShiftded=r*NumColsImg+c;
      imageTile[offsetRows+threadIdx.y][offsetCols+threadIdx.x] = image[vIdxShiftded];
    }


    // ---FILL TILE BORDERS---
    // ---FILL TILE BORDERS---
    // ---FILL TILE BORDERS---
    // ---FILL TILE BORDERS---
    // ---FILL TILE BORDERS---
    
    // Fills the rows above the block in the tile
    // Uses the first (offsetRows) rows of threads to copy
    if (threadIdx.y < offsetRows) {
      int aboveIdxY = idxY - offsetRows;
      if (aboveIdxY < 0) {
        aboveIdxY = 0;
      }
      int aboveIdxX = idxX;
      if (aboveIdxX >= NumColsImg) {
        aboveIdxX = NumColsImg - 1;
      }
      //calculate the index in the original matrix
      int aboveVIdxShiftded=aboveIdxY*NumColsImg+aboveIdxX;
      imageTile[threadIdx.y][offsetCols+threadIdx.x] = image[aboveVIdxShiftded];
    }

    // Fills the cols left of the block in the tile
    // Uses the first (offsetCols) cols of threads to copy
    if (threadIdx.x < offsetCols) {
      int leftIdxY = idxY;
      if (leftIdxY >= NumRowsImg) {
        leftIdxY = NumRowsImg - 1;
      }
      int leftIdxX = idxX - offsetCols;
      if (leftIdxX < 0) {
        leftIdxX = 0;
      }
      //calculate the index in the original matrix
      int leftVIdxShiftded=leftIdxY*NumColsImg+leftIdxX;
      imageTile[threadIdx.y+offsetRows][threadIdx.x] = image[leftVIdxShiftded];
    }

    // Fills the rows below the block in the tile
    // Uses the last (offsetRows) rows of threads to copy
    if (threadIdx.y+offsetRows >= blockDim.y) {
      int belowIdxY = idxY + offsetRows;
      if (belowIdxY >= NumRowsImg) {
        belowIdxY = NumRowsImg - 1;
      }

      int belowIdxX = idxX;
      if (belowIdxX >= NumColsImg) {
        belowIdxX = NumColsImg - 1;
      }
      //calculate the index in the original matrix
      int belowVIdxShifted=belowIdxY*NumColsImg+belowIdxX;
      imageTile[threadIdx.y+2*offsetRows][offsetCols+threadIdx.x] = image[belowVIdxShifted];
    }

    // Fills the cols right of the block in the tile
    // Uses the last (offsetCols) cols of threads to copy
    if (threadIdx.x +offsetCols>= blockDim.x ) {
      int rightIdxY = idxY;
      if (rightIdxY >= NumRowsImg) {
        rightIdxY = NumRowsImg - 1;
      }
      int rightIdxX = idxX + offsetCols;
      if (rightIdxX >= NumColsImg) {
        rightIdxX = NumColsImg - 1;
      }
      //calculate the index in the original matrix
      int rightVIdxShiftded=rightIdxY*NumColsImg+rightIdxX;
      imageTile[threadIdx.y+offsetRows][2*offsetCols+threadIdx.x] = image[rightVIdxShiftded];
    }



    // ---FILL TILE CORNERS---
    // ---FILL TILE CORNERS---
    // ---FILL TILE CORNERS---
    // ---FILL TILE CORNERS---
    // ---FILL TILE CORNERS---

    //upper left corner
    if ((threadIdx.y<offsetRows)&&(threadIdx.x<offsetCols)){
      int upperLeftIdxY=idxY-offsetRows;
      //check if the corner is above the first row
      if (upperLeftIdxY<0){
        upperLeftIdxY=0;
      } 
      //check if the corner is out of the matrix(start left column)
      int upperLeftIdxX=idxX-offsetCols;
      if(upperLeftIdxX<0){
        upperLeftIdxX=0;
      } 
      //calculate the index in the original matrix
       int upperLeftVIdxShiftded=upperLeftIdxY*NumColsImg+upperLeftIdxX;
      imageTile[threadIdx.y][threadIdx.x]=image[upperLeftVIdxShiftded];
    }


    //upper right corner
    if ((threadIdx.y<offsetRows)&&(threadIdx.x+offsetCols >= blockDim.x )){
      int upperRightIdxY=idxY-offsetRows;
      //check if the corner is above the first row
      if (upperRightIdxY<0){
        upperRightIdxY=0;
      }
      //check if corner is out of the matrix(right column)
      int upperRightIdxX=idxX;
      if(upperRightIdxX>=NumColsImg){
        upperRightIdxX=NumColsImg-1;
      }
      //calculate the index in the original matrix
      int upperRightVIdxShiftded=upperRightIdxY*NumColsImg+upperRightIdxX;
      imageTile[threadIdx.y][threadIdx.x+2*offsetCols]=image[upperRightVIdxShiftded];
    }


    //below right corner
    if ((threadIdx.y+offsetRows>= blockDim.y)&&(threadIdx.x+offsetCols >= blockDim.x )){
      int belowRightIdxY=idxY+offsetRows;
      //check if the corner is out of the matrix(under)
      if (belowRightIdxY>=NumRowsImg){
        belowRightIdxY=NumRowsImg-1;
      }
      //check if corner is out of the matrix(right column)
      int belowRightIdxX=idxX+offsetCols;
      if(belowRightIdxX>=NumColsImg){
        belowRightIdxX=NumColsImg-1;
      } 
      //calculate the index in the original matrix
      int belowRightVIdxShiftded=belowRightIdxY*NumColsImg+belowRightIdxX;
      imageTile[threadIdx.y+2*offsetRows][threadIdx.x+2*offsetCols]=image[belowRightVIdxShiftded];
    }


    //below left corner
    if ((threadIdx.y+offsetRows>= blockDim.y)&&(threadIdx.x<offsetCols )){
      int belowLeftIdxY=idxY+offsetRows;
      //check if the corner is out of the matrix(under)
      if (belowLeftIdxY>=NumRowsImg){
        belowLeftIdxY=NumRowsImg-1;
      }
      //check if the corner is out of the matrix(start left column)
      int belowLeftIdxX=idxX-offsetCols;
      if(belowLeftIdxX<0){
        belowLeftIdxX=0;
      } 
      //calculate the index in the original matrix
      int belowLeftVIdxShiftded=belowLeftIdxY*NumColsImg+belowLeftIdxX;
      imageTile[threadIdx.y+2*offsetRows][threadIdx.x]=image[belowLeftVIdxShiftded];
    }




    // ---FILL FILTER---
    // ---FILL FILTER---
    // ---FILL FILTER---
    // ---FILL FILTER---
    // ---FILL FILTER---

    // threads populate filterShared
    if( (threadIdx.x<NumColsFilter) && (threadIdx.y<NumRowsFilter)){
      filterShared[threadIdx.y][threadIdx.x]=filter_mask[threadIdx.y*NumColsFilter+threadIdx.x];
    }
    

    // ---APPLY FILTER---
    // ---APPLY FILTER---
    // ---APPLY FILTER---
    // ---APPLY FILTER---
    // ---APPLY FILTER---

    //wait till all thread finishes to fill the tile
    __syncthreads();

    //  compute the filtered pixel value and store it in filtered_image[vX]
    float accu = 0;
    int y, x;
    //image convolution
    //check if i'm inside the final matrix
    if ( ( ( idxY ) < NumRowsImg ) && ( ( idxX ) < NumColsImg )){
      for (y = 0; y < NumRowsFilter; y++){
         for (x = 0; x < NumColsFilter; x++){
           //calculate the filtered value for a pixel
            accu += imageTile[threadIdx.y + y][threadIdx.x + x] * filterShared[y][x];
            
         }
      }
      //store the value inside the filtered matrix
      filtered_image[idxY*NumColsImg+idxX] = accu;
    }
      // Synchronize to make sure that the preceding computation is done 
    __syncthreads();



}

"""



In [ ]:
BLOCK_SIZE=16

In [ ]:
FILTER_ROWS=filter_mask.shape[0]
FILTER_COLS=filter_mask.shape[1]

In [ ]:
Image_filter_src=image_filter_tmp%{
    'BLOCK_SIZE':BLOCK_SIZE,
    'FILTER_ROWS':FILTER_ROWS,
    'FILTER_COLS':FILTER_COLS


}

In [ ]:
kernel = SourceModule(Image_filter_src)

And import the kernel definition to python

In [ ]:
image_filter = kernel.get_function ('image_filter')

##Prepares the execution environment 

Allocate the temporal filtered memory layer, where will be store the filtered image.

In [ ]:
filtered_image_layer_gpu = gpuarray.zeros(image_shape,np.float32)

Here defines the main block.
Walk through the layers. Copying the selected layer to temporal image_layer, upload the layer to the GPU and invokes the image_filter kernel.

In [ ]:
start_t = time.time()
for layer in range(depth):
  image_layer = np.zeros(image_shape,dtype=np.float32) #Allocate the temporal image layer
  image_layer=(image[:,:,layer]).astype(np.float32)    #Copy from the orignal image the selected layer into the temporal memory
  image_layer_gpu=gpuarray.to_gpu(image_layer)         #Uploads the image layer to the GPU
  filtered_image_layer_gpu.fill(0.0)                   #Fills with 0 the target memory in the GPU
  image_filter(                                        #invokes the kernel
          image_layer_gpu,
          filter_mask_gpu,
          filtered_image_layer_gpu,
          np.int32(image_shape[0]),
          np.int32(image_shape[1]),
          np.int32(filter_mask.shape[0]),
          np.int32(filter_mask.shape[1]),
          block=block_size,
          grid=grid_size
        )
  filtered_image_layer=filtered_image_layer_gpu.get() #Download from the GPU the filtered image
  result_image[:,:,layer] = filtered_image_layer.astype(np.uint8) #Copy the filtered image to the final image store, 
                                                                  #casting the data type from double to uint8
end_t = time.time()

#Show results


In [ ]:
print("Execution time is: {0} seconds.".format(end_t-start_t))

In [ ]:
plt.figure()
plt.imshow(result_image)

## **ANALISYS AND CONCLUSION**

#### **TIME COMPARISON CPU VS GPU :**

We can compare two different implementation of image filter using the `first practical work` that was implemented using **multiprocessing** and this work that was implemented using **pyCuda library**.

IMAGE1(chess.jpg):

- *CPU* time using `filter1`(3x3): Execution time is: ***3.6 seconds***

- *CPU* time using `filter2`(1x3): Execution time is: ***2.5 seconds***

- *GPU* time using `filter1`(3x3): Execution time is: ***0.0032 seconds.***

- *GPU* time using `filter2`(1x3): Execution time is: ***0.0031 seconds.***


We can notice that filtering using *GPU* is a lot more fast than *CPU* infact comparing the results of `filter1` is approximately ***1093.75 times faster***

Also using *CPU* `filter1` is **1.5 time slower than `filter2`** that's because of less computation done by the *CPU*  due to the difference of size of the filters.

Instead for the *GPU* the results are not as may expected infact the difference of time between `filter1` and `filter2` is almost the same ,even if the filter size between the two is different, that's because of the really high computational power of the *GPU*.

So we can conclude saying that because the *GPU* is much faster (approximately 1093 times in this case) than the *CPU* filtering an image with the *GPU* is a much better and faster approach than filtering with the *CPU*.




#### **SHARED MEMORY VS GLOBAL MEMORY**

In this work we used the shared memory tile approach instead of using global memory in the *GPU* but what of the two approach is the better one?

as we saw in other works accessing to **shared memory** is much faster 
than **global** due to the fact that's shared memory is faster to access. 

However, in order to do anything you need to fill the **shared memory** with something, you usually pull from the **global memory**, in this case we had to **fill the tile of the shared memory** with the value in the image in the global memory but we have also to fill the borders and corners to have the computation of the filter working well.

The use of **shared memory** is when you need to within a block of threads, reuse data already pulled or evaluated from **global memory**. So instead of pulling from global memory again, you put it **in the shared memory for other threads within the same block to see and reuse.**









